<a href="https://colab.research.google.com/github/AryaFeng/pytorch/blob/main/01/pytorch_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# pytorch 同时也提供了torchtext,torchVision,torchAudio的数据集

其中torchvision.datasets模块红有一个类Dataset,包含了很多的数据集，如CIFAR，COCO等。每一个TorchVision包含了两个参数，transoform和target_transform用来调整修改样本和标签。例如进行裁剪等

In [3]:
# download dataset
train_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)
# download test
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:01<00:00, 17943998.17it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 272755.26it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5042581.96it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 20841966.21it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



Dataset会作为参数传递给DataLoader，Dataloader支持自动批量、采样、打乱、多线程数据加载。

In [4]:
batch_size = 64

train_dataloader = DataLoader(train_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

for X,y in test_dataloader:
  print(f"shapeof X [N,C,,H,W]:{X.shape}")
  print(f"shapeof y :{y.shape} {y.dtype}")
  break

shapeof X [N,C,,H,W]:torch.Size([64, 1, 28, 28])
shapeof y :torch.Size([64]) torch.int64


创建模型

In [5]:
device =( "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu" )
print(device)


# define model
class NerualNetwork(nn.Module):
  def __init__(self):



cuda
